In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
model = BertForSequenceClassification.from_pretrained("../model/classifier_model")

sentence = ""

token = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
output = model(**token)
pred = torch.argmax(output.logits, dim=1)
print(pred)

tensor([4])
